In [1]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shapley import ShapleyIndices, ShapleyKrigingIndices, SobolIndices, SobolKrigingIndices
from shapley.tests import Ishigami, AdditiveGaussian
from shapley.tests.utils import true_gaussian_full_ind_sobol
from shapley.plots import set_style_paper, plot_sensitivity_results
from shapley.plots import plot_correlation_indices

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

savefigs = False
ylim = [-0.05, 1.05]

In [2]:
dim = 3
beta = None
model = AdditiveGaussian(dim=dim, beta=beta)
model.margins = [ot.Normal()]*(dim-1) + [ot.Normal(0, 2.)]

In [84]:
ot.RandomGenerator.SetSeed(0)
np.random.seed(0)

Nv = 1000
No = 20
n_test = 10
min_Ni = 5
max_Ni = 10
n_boot = 100

In [85]:
all_Ni = np.arange(min_Ni, max_Ni, dtype=int)
n_Ni = len(all_Ni)
all_results = np.zeros((n_Ni, dim, n_boot))
for i_test in range(n_test):
    for i, Ni in enumerate(all_Ni):
        # Shapley
        shapley = ShapleyIndices(model.input_distribution)
        shapley.build_mc_sample(model=model, n_perms=None, Nv=Nv, No=No, Ni=Ni)
        shapley_results = shapley.compute_indices(n_boot=n_boot)
        all_results[i] = shapley_results.full_shapley_indices

In [67]:
no_boot_estimation = all_results[:, :, 0]
quantiles = np.percentile(all_results[:, :, 1:], [2.5, 97.5], axis=2)
error_up = 2*no_boot_estimation - quantiles[0]
error_down = 2*no_boot_estimation - quantiles[1]

In [82]:
true_indices = model.shapley_indices
(error_down < true_indices) & (error_up > true_indices)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True, False]], dtype=bool)